In [5]:
from examples.util.getDataset import getDataset
from examples.util.plotting import plot_w
from examples.util.test_fns import *
from examples.util.testing import shrimp_v_random, growth
from examples.util.metrics import shrimp_test, make_l2_loss
from sparse_rf.util import *
from sparse_rf.modules import make_A, make_W
from sparse_rf.algs import shrimp, l2, l1, sindy
from functools import partial
import matplotlib.pyplot as plt
from math import ceil
import gc

In [6]:
r = 9
fn = partial(fn9, r=r)
q = 1
d = 100
m = 200
N = 1500
l = 1e-13

Xtr, Xte, Ytr, Yte = getDataset(fn, d=d, m=m)

trials = 10

scales = np.arange(1/q, 206, 5)

In [8]:
for scale_ in scales:
    svr_q = shrimp_v_random(trials, Xtr, Ytr, Xte, Yte, q, N, l, scale=np.sqrt(scale_))
    errs_bp = []
    for _ in range(trials):
        W, _ = make_W(d, q, N, scale=1/np.sqrt(scale_))
        Atr = make_A(Xtr, W)
        Ate = make_A(Xte, W)
        c = l1(Atr, Ytr)
        Ypred = Ate@c
        err_bp = np.sum((Ypred-Yte)**2)/len(Yte)

        errs_bp.append(err_bp)

        del W
        del Atr
        del Ate
        del c
        del Ypred

        gc.collect()

    print("Variance: {}".format(scale_))
    print("Min l2: {}".format(svr_q.err_l2))
    print("SHRIMP: {}".format(svr_q.err_shrimp))
    print("Random pruning: {}".format(svr_q.err_r))
    print("Min l1: {}".format(np.mean(errs_bp)))
    print("----------------------------")

    del errs_bp
    del svr_q

    gc.collect()

Variance: 1.0
Min l2: 137.7344945981377
SHRIMP: 43.29805160272907
Random pruning: 44.181665818003054
Min l1: 138.25345705899701
----------------------------
Variance: 6.0
Min l2: 70.84909082859386
SHRIMP: 40.6738028676489
Random pruning: 45.323578608445224
Min l1: 155.91021610834704
----------------------------
Variance: 11.0
Min l2: 59.36994340530627
SHRIMP: 16.32944786722441
Random pruning: 44.7890920930049
Min l1: 113.95003548965103
----------------------------
Variance: 16.0
Min l2: 51.479086899259435
SHRIMP: 19.62893474427543
Random pruning: 38.838140891997085
Min l1: 105.39850319364447
----------------------------
Variance: 21.0
Min l2: 47.054245678780056
SHRIMP: 6.061726588972201
Random pruning: 40.83921382915271
Min l1: 102.02403579351775
----------------------------
Variance: 26.0
Min l2: 48.063952018536725
SHRIMP: 7.276848459963513
Random pruning: 45.33342204797908
Min l1: 102.52194110832303
----------------------------
Variance: 31.0
Min l2: 42.99613095508451
SHRIMP: 8.90431